In [ ]:
#hyperparameter tuning

best_auc=0
best_param={0,0,0,0,0,0}
all_auc=[]
all_param=[]
best_aucpr=0
all_aucpr=[]
best_parampr={0,0,0,0,0,0}
time=[]
confusion=[]

start=datetime.now()

for i in range(16):
     for k in range(20):
        
        print('run ' + str(i) + '.' + str(k) + ' started at ' + str(datetime.now()))
        np.random.seed(i)
        train_df_split = train_df_ready.reindex(np.random.permutation(train_df_ready.index))
        val_ids = list(train_df_split['EC batch'].unique())[-int(val_ratio*len(train_df_split['EC batch'].unique())):]
        val_df_pre = train_df_ready[train_df_ready['EC batch'].isin(val_ids)]
        train_df_pre = train_df_ready[~train_df_ready['EC batch'].isin(val_ids)]
        
        del train_df_split

        n_estimators = np.random.randint(150, 1000)
        learning_rate = np.random.uniform(0.01, 0.07)
        subsample = np.random.uniform(0.1, 1)
        max_depth = np.random.randint(3, 9)
        colsample_bytree = np.random.uniform(0.1, 1)
        min_child_weight = np.random.randint(1, 4)
        params_str = "_".join([str(n_estimators), str(learning_rate), str(subsample), str(max_depth), 
                               str(colsample_bytree), str(min_child_weight)])


        X_train = train_df_pre.drop(['Adherence','Effective Dating','unique_id','unique_id_event','create_timestamp','EC batch'],axis=1)
        y_train = train_df_pre['Adherence']
        X_val = val_df_pre.drop(['Adherence','Effective Dating','unique_id','unique_id_event','create_timestamp','EC batch'],axis=1)
        y_val = val_df_pre['Adherence']
        class_ratio=(y_train.value_counts()[0]/y_train.value_counts()[1])
        
        del train_df_pre,val_df_pre

#         class_ratio=9
        
        xgb_model= xgb.XGBClassifier(objective='binary:logistic',n_estimators=n_estimators,learning_rate=learning_rate,subsample=subsample,
                                    max_depth=max_depth,colsample_bytree=colsample_bytree,min_child_weight=min_child_weight,eval_metric='auc',
                                    use_label_encoder=False,random_state=(i+1)*(k+1), scale_pos_weight=class_ratio, tree_method='hist')

#         print(datetime.now())
        xgb_model.fit(X_train, y_train)
#         print(datetime.now())
        ns_probs = [0 for _ in range(len(y_val))]

        lr_probs = xgb_model.predict_proba(X_val)
        # keep probabilities for the positive outcome only
        lr_probs = lr_probs[:, 1]
        
        # calculate the precision-recall auc
        precision, recall, _ = precision_recall_curve(y_val, lr_probs)
        pr_auc = auc(recall, precision)
#         print(pr_auc)
        
        # calculate scores
        ns_auc = roc_auc_score(y_val, ns_probs)
        lr_auc = roc_auc_score(y_val, lr_probs)
        # summarize scores
#         print(lr_auc)
        predictions = xgb_model.predict(X_val)
#         print(classification_report(y_val,predictions))
        
        confusion.append(precision_recall_fscore_support(y_val, predictions, average=None))
        confusion.append(precision_recall_fscore_support(y_val, predictions, average='macro'))
        confusion.append(precision_recall_fscore_support(y_val, predictions, average='weighted'))
        
        if lr_auc>best_auc:
            best_auc=lr_auc
            best_param={'run ' +str(i)+'.'+str(k), 'n_estimatmors' +str(n_estimators), 'lr' + str(learning_rate),
                        'ss' + str(subsample), 'md' + str(max_depth) , 'cs' + str(colsample_bytree) , 'mcw' + str(min_child_weight)}
        
        if pr_auc>best_aucpr:
            best_aucpr=pr_auc
            best_parampr={'run ' +str(i)+'.'+str(k), 'n_estimatmors' +str(n_estimators), 'lr' + str(learning_rate),
                        'ss' + str(subsample), 'md' + str(max_depth) , 'cs' + str(colsample_bytree) , 'mcw' + str(min_child_weight)}

        all_auc.append(lr_auc)
        all_aucpr.append(pr_auc)
        all_param.append(['n_estimators= ' +str(n_estimators),'learning_rate= ' +str(learning_rate),
                          'subsample= ' +str(subsample), 'max_depth= ' +str(max_depth), 'colsample_bytree= ' +str(colsample_bytree),
                          'min_child_weight= ' +str(min_child_weight)])
        
        
        print(pr_auc)
        print(lr_auc)
        print('run ' + str(i)  + '.' + str(k) + ' ended at ' + str(datetime.now()))
        
print('cv ended at ' + str(datetime.now()))

end=datetime.now()
time.append(end-start)

fileexport_confusion='Data/confusion_xgb_agg.csv'
fileexport_time='Data/time_xgb_agg.csv'
fileexport_bestauc='Data/best_auc_xgb_agg.csv'
fileexport_bestparam='Data/best_param_xgb_agg.csv'
fileexport_bestaucpr='Data/best_aucpr_xgb_agg.csv'
fileexport_bestparampr='Data/best_parampr_xgb_agg.csv'
fileexport_auc='Data/all_auc_xgb_agg.csv'
fileexport_aucpr='Data/all_aucpr_xgb_agg.csv'
fileexport_param='Data/all_param_xgb_agg.csv'
pd.DataFrame(best_param).to_csv(fileexport_bestparam,index=False, header=False)
pd.DataFrame(best_parampr).to_csv(fileexport_bestparampr,index=False, header=False)
pd.DataFrame(all_auc).to_csv(fileexport_auc,index=False, header=False)
pd.DataFrame(all_aucpr).to_csv(fileexport_aucpr,index=False, header=False)
pd.DataFrame(all_param).to_csv(fileexport_param,index=False, header=False)
pd.DataFrame(time).to_csv(fileexport_time,index=False, header=False)
pd.DataFrame(confusion).to_csv(fileexport_confusion, index=False, header=False)

In [ ]:
best_auc=0
best_param={0,0,0,0,0,0}
all_auc=[]
all_param=[]
best_aucpr=0
all_aucpr=[]
best_parampr={0,0,0,0,0,0}
time=[]
confusion=[]

start=datetime.now()

for i in range(16):
     for k in range(20):
        
        print('run ' + str(i) + '.' + str(k) + ' started at ' + str(datetime.now()))
        np.random.seed(i)
        train_df_split = train_df_ready.reindex(np.random.permutation(train_df_ready.index))
        val_ids = list(train_df_split['EC batch'].unique())[-int(val_ratio*len(train_df_split['EC batch'].unique())):]
        val_df_pre = train_df_ready[train_df_ready['EC batch'].isin(val_ids)]
        train_df_pre = train_df_ready[~train_df_ready['EC batch'].isin(val_ids)]
        
        del train_df_split

        n_estimators = np.random.randint(150, 1000)
        learning_rate = np.random.uniform(0.01, 0.07)
        subsample = np.random.uniform(0.1, 1)
        max_depth = np.random.randint(3, 9)
        colsample_bytree = np.random.uniform(0.1, 1)
        min_child_weight = np.random.randint(1, 4)
        params_str = "_".join([str(n_estimators), str(learning_rate), str(subsample), str(max_depth), 
                               str(colsample_bytree), str(min_child_weight)])


        X_train = train_df_pre.drop(['Adherence','Effective Dating','unique_id','unique_id_event','create_timestamp','EC batch'],axis=1)
        y_train = train_df_pre['Adherence']
        X_val = val_df_pre.drop(['Adherence','Effective Dating','unique_id','unique_id_event','create_timestamp','EC batch'],axis=1)
        y_val = val_df_pre['Adherence']
        class_ratio=(y_train.value_counts()[0]/y_train.value_counts()[1])
        
        del train_df_pre,val_df_pre

#         class_ratio=9
        
        xgb_model= xgb.XGBClassifier(objective='binary:logistic',n_estimators=n_estimators,learning_rate=learning_rate,subsample=subsample,
                                    max_depth=max_depth,colsample_bytree=colsample_bytree,min_child_weight=min_child_weight,eval_metric='auc',
                                    use_label_encoder=False,random_state=(i+1)*(k+1), scale_pos_weight=class_ratio, tree_method='hist')

#         print(datetime.now())
        xgb_model.fit(X_train, y_train)
#         print(datetime.now())
        ns_probs = [0 for _ in range(len(y_val))]

        lr_probs = xgb_model.predict_proba(X_val)
        # keep probabilities for the positive outcome only
        lr_probs = lr_probs[:, 1]
        
        # calculate the precision-recall auc
        precision, recall, _ = precision_recall_curve(y_val, lr_probs)
        pr_auc = auc(recall, precision)
#         print(pr_auc)
        
        # calculate scores
        ns_auc = roc_auc_score(y_val, ns_probs)
        lr_auc = roc_auc_score(y_val, lr_probs)
        # summarize scores
#         print(lr_auc)
        predictions = xgb_model.predict(X_val)
#         print(classification_report(y_val,predictions))
        
        confusion.append(precision_recall_fscore_support(y_val, predictions, average=None))
        confusion.append(precision_recall_fscore_support(y_val, predictions, average='macro'))
        confusion.append(precision_recall_fscore_support(y_val, predictions, average='weighted'))
        
        if lr_auc>best_auc:
            best_auc=lr_auc
            best_param={'run ' +str(i)+'.'+str(k), 'n_estimatmors' +str(n_estimators), 'lr' + str(learning_rate),
                        'ss' + str(subsample), 'md' + str(max_depth) , 'cs' + str(colsample_bytree) , 'mcw' + str(min_child_weight)}
        
        if pr_auc>best_aucpr:
            best_aucpr=pr_auc
            best_parampr={'run ' +str(i)+'.'+str(k), 'n_estimatmors' +str(n_estimators), 'lr' + str(learning_rate),
                        'ss' + str(subsample), 'md' + str(max_depth) , 'cs' + str(colsample_bytree) , 'mcw' + str(min_child_weight)}

        all_auc.append(lr_auc)
        all_aucpr.append(pr_auc)
        all_param.append(['n_estimators= ' +str(n_estimators),'learning_rate= ' +str(learning_rate),
                          'subsample= ' +str(subsample), 'max_depth= ' +str(max_depth), 'colsample_bytree= ' +str(colsample_bytree),
                          'min_child_weight= ' +str(min_child_weight)])
        
        
        print(pr_auc)
        print(lr_auc)
        print('run ' + str(i)  + '.' + str(k) + ' ended at ' + str(datetime.now()))
        
print('cv ended at ' + str(datetime.now()))

end=datetime.now()
time.append(end-start)

fileexport_confusion='Data/confusion_xgb_agg_pca.csv'
fileexport_time='Data/time_xgb_agg_pca.csv'
fileexport_bestauc='Data/best_auc_xgb_agg_pca.csv'
fileexport_bestparam='Data/best_param_xgb_agg_pca.csv'
fileexport_bestaucpr='Data/best_aucpr_xgb_agg_pca.csv'
fileexport_bestparampr='Data/best_parampr_xgb_agg_pca.csv'
fileexport_auc='Data/all_auc_xgb_agg_pca.csv'
fileexport_aucpr='Data/all_aucpr_xgb_agg_pca.csv'
fileexport_param='Data/all_param_xgb_agg_pca.csv'
pd.DataFrame(best_param).to_csv(fileexport_bestparam,index=False, header=False)
pd.DataFrame(best_parampr).to_csv(fileexport_bestparampr,index=False, header=False)
pd.DataFrame(all_auc).to_csv(fileexport_auc,index=False, header=False)
pd.DataFrame(all_aucpr).to_csv(fileexport_aucpr,index=False, header=False)
pd.DataFrame(all_param).to_csv(fileexport_param,index=False, header=False)
pd.DataFrame(time).to_csv(fileexport_time,index=False, header=False)
pd.DataFrame(confusion).to_csv(fileexport_confusion, index=False, header=False)



 